# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B 를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A:산, B:바다, C:나무, D:물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ? ] <br>
모델이 예측한 가장 적합한 단어: 잠입 <br>
당신의 답변과 모델 예측의 유사도: 0.34 <br>
아쉽네요. 더 생각해보세요.

In [ ]:
# !pip install Korpora


   ------------- -------------------------- 1/3 [xlrd]
   -------------------------- ------------- 2/3 [Korpora]
   ---------------------------------------- 3/3 [Korpora]



In [11]:
from Korpora import Korpora
Korpora.fetch('korean_petitions')

corpus = Korpora.load('korean_petitions')

[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-09
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-10
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-11
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2017-12
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2018-01
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2018-02
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\shin\Korpora\korean_petitions\petitions_2018-03
[Korpora] Corpus `korean_petitions` is already installed at C:\U

In [12]:
from gensim.models import FastText, Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import kss
from konlpy.tag import Okt
import string
import re
import random

In [4]:
corpus

In [33]:
sen = list(corpus.train.texts)[:100000] # + corpus.train.pairs) + list(corpus.dev.texts + corpus.dev.pairs) + list(corpus.test.texts + corpus.test.pairs)
len(sen)

100000

In [15]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    
    return stopwords

In [16]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [34]:
okt = Okt()
preprocessed_sentences = []
ko_stopwords = load_stopwords('ko_stopwords.txt')

for s in sen:
    s = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', s)
    tokens = okt.pos(s, stem=True)
    cleaned_tokens = [token for token, pos in tokens if token not in ko_stopwords and token not in punct and pos in ['Noun', 'Adjective']]

    preprocessed_sentences.append(cleaned_tokens)

len(preprocessed_sentences)

100000

### 오류!

**sentences에는 각 sentence별로 나눠진 list를 넣어야됨 2차원 배열**

In [35]:
fasttext_model = FastText(
    sentences=preprocessed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1
)

# vector_size 조정
# window 크기 조정
# sg 1로 바꿔보기

In [36]:
fasttext_model.wv.index_to_key

['이다',
 '없다',
 '아니다',
 '국민',
 '수',
 '생각',
 '사람',
 '법',
 '정부',
 '많다',
 '말',
 '대통령',
 '더',
 '대한민국',
 '위해',
 '위',
 '경찰',
 '국가',
 '문제',
 '그렇다',
 '지금',
 '대한',
 '청원',
 '나라',
 '좋다',
 '때문',
 '범죄',
 '사건',
 '돈',
 '이렇다',
 '경우',
 '사회',
 '화폐',
 '현재',
 '청소년',
 '폐지',
 '안되다',
 '정책',
 '이유',
 '처벌',
 '부패하다',
 '민원',
 '어떻다',
 '대해',
 '전',
 '학생',
 '분',
 '사용',
 '가상',
 '문재인',
 '정말',
 '상황',
 '관련',
 '한국',
 '보호',
 '우리나라',
 '인하다',
 '번',
 '글',
 '못',
 '피해자',
 '모든',
 '공무원',
 '안',
 '명',
 '불법',
 '피해',
 '학교',
 '알',
 '시',
 '교육',
 '후',
 '필요하다',
 '만원',
 '사실',
 '지원',
 '제도',
 '정도',
 '내용',
 '중',
 '집',
 '여성',
 '곳',
 '내',
 '북한',
 '다시',
 '부탁드리다',
 '발생',
 '요',
 '가지',
 '세금',
 '살',
 '현실',
 '결정',
 '회사',
 '개인',
 '반대',
 '조직',
 '해결',
 '보고',
 '제발',
 '비',
 '야하다',
 '지역',
 '인권',
 '관리',
 '입',
 '이명박',
 '계획',
 '조사',
 '개',
 '통해',
 '기관',
 '사고',
 '이제',
 '날',
 '관',
 '기사',
 '가족',
 '이용',
 '앞',
 '규제',
 '투자',
 '차',
 '당하다',
 '운영',
 '보험',
 '청와대',
 '서민',
 '계속',
 '꼭',
 '가장',
 '근무',
 '사업',
 '점',
 '미래',
 '의무',
 '방법',
 '시행',
 '제대로',
 '한번',
 '개선',
 '마음',
 '본

In [54]:
fasttext_model.wv.save_word2vec_format('game_w2v')

In [55]:
fasttext_model.save('game_fasttext.model')

In [3]:
fasttext_model = FastText.load('game_fasttext.model')

In [53]:
# print(A)
threshold = 0.8
filtered = []
while not filtered:
    A = random.choice(fasttext_model.wv.index_to_key)
    filtered = [word for word, score in fasttext_model.wv.most_similar(A, topn=1000) if score >= threshold]

B = random.choice(filtered)
sim_ab = fasttext_model.wv.similarity(A, B)

C = A
filtered = []
while C == A or C == B or (not filtered):
    C = random.choice(fasttext_model.wv.index_to_key)
    filtered = [(word, score) for word, score in fasttext_model.wv.most_similar(C, topn=1000) if score >= threshold]

print(f"관계 [ {A} : {B} = {C} : ? ]")

D = input()

sim_user = fasttext_model.wv.similarity(C, D)
model_out = ''
sim_model = 0
for model_choice in filtered:
    if abs(model_choice[1]-sim_ab) < abs(sim_model - sim_ab):
        sim_model = model_choice[1]
        model_out = model_choice[0]

print(f"모델이 예측한 단어 : {model_out}")
print(f"입력한 답: {D}")
print(f"당신의 답변과 모델 예측의 유사도: {fasttext_model.wv.similarity(D, model_out)}")


관계 [ 엠비씨 : 구본홍 = 장주 : ? ]
모델이 예측한 단어 : 장상
입력한 답: 장장
당신의 답변과 모델 예측의 유사도: 0.6295713782310486


In [41]:
print(f"{A}와 {B}의 유사도: {sim_ab}")
print(f"{C}와 {model_out}의 유사도: {sim_model}")
print(f"{C}와 {D}의 유사도: {sim_user}")
print(filtered)

폐쇄할거와 왜안의 유사도: 0.8375065326690674
축구단와 브라질월드컵의 유사도: 0.8381701707839966
축구단와 지단의 유사도: 0.10404076427221298
[('축구팀', 0.9219532608985901), ('상주상무', 0.8782293200492859), ('프리미어리그', 0.8685798048973083), ('한일월드컵', 0.8667595386505127), ('핫스퍼', 0.8660152554512024), ('토트넘', 0.8643359541893005), ('축구인', 0.8640791177749634), ('피겨스케이팅', 0.8544263243675232), ('프로축구', 0.8537127375602722), ('우승하다', 0.8472102880477905), ('스포츠조선', 0.8452106714248657), ('자카르타', 0.8419578075408936), ('골키퍼', 0.8402000665664673), ('야구단', 0.8392343521118164), ('러시아월드컵', 0.8381751775741577), ('브라질월드컵', 0.8381701707839966), ('구단', 0.8363161087036133), ('김연경', 0.8327764868736267), ('차범근', 0.8316003084182739), ('아시안컵', 0.8288240432739258), ('피겨', 0.8279991745948792), ('스케이팅', 0.8277334570884705), ('동계체전', 0.8273382186889648), ('박찬호', 0.8264262676239014), ('양궁', 0.8232324123382568), ('국가대표팀', 0.8219887018203735), ('태릉선수촌', 0.8206746578216553), ('프리미어', 0.8198981881141663), ('챔피언스', 0.8186010718345642), ('테니스', 0.8162599205970764)